### 1.多个文件合并成一个文件

In [45]:
import pandas as pd
from sqlalchemy import create_engine 

In [46]:
'''create_engine()用来初始化数据库连接，括号内为连接信息：
（“数据库类型+数据库驱动名称：//用户名：密码@数据库服务器的名称或IP地址：端口号/
数据库名称?driver=SQL+Server+Native+Client+11.0',encoding='utf-8',echo=False,poolclass=NullPool）'''

engine = create_engine('mssql+pyodbc://:@CLAIRE/baidu_poi?driver=SQL server')

In [47]:
 engine.table_names() 

['baidu_poi_房地产',
 'baidu_poi_公司企业',
 'baidu_poi_购物',
 'baidu_poi_交通设施',
 'baidu_poi_教育培训',
 'baidu_poi_金融',
 'baidu_poi_酒店',
 'baidu_poi_丽人',
 'baidu_poi_旅游景点',
 'baidu_poi_美食',
 'baidu_poi_汽车服务',
 'baidu_poi_生活服务',
 'baidu_poi_文化传媒',
 'baidu_poi_休闲娱乐',
 'baidu_poi_医疗',
 'baidu_poi_运动健身',
 'baidu_poi_政府机构']

In [48]:
def concat_file(table):
    with engine.connect() as conn,conn.begin():
        data = pd.read_sql_table(table,conn)
        return data

In [49]:
df_list = [concat_file(poi) for poi in engine.table_names()]

In [50]:
all_data = pd.concat(df_list)

In [51]:
all_data.head()

,index,name,lat,lng,sort1,sort2
0,0,丰泽科技园-2号楼,34.176428,108.816795,房地产,写字楼
1,1,高新新区,34.173326,108.827928,房地产,写字楼
2,0,丰泽科技园-5号楼,34.175525,108.817244,房地产,住宅区
3,1,丰泽科技路3号楼,34.175989,108.817210,房地产,住宅区
4,2,鑫垚新区,34.174688,108.815573,房地产,住宅区


In [52]:
len(all_data)

244961

### 2.删除重复项

In [53]:
all_data01 = all_data.drop_duplicates(subset = ['name','lat','lng'])

In [54]:
len(all_data01)

169792

In [60]:
all_data01.head()

,index,name,lat,lng,sort1,sort2
0,0,丰泽科技园-2号楼,34.176428,108.816795,房地产,写字楼
1,1,高新新区,34.173326,108.827928,房地产,写字楼
2,0,丰泽科技园-5号楼,34.175525,108.817244,房地产,住宅区
3,1,丰泽科技路3号楼,34.175989,108.817210,房地产,住宅区
4,2,鑫垚新区,34.174688,108.815573,房地产,住宅区


#### 3.删除范围之外的数据
34.16985,108.756009,34.395586,109.146952

In [70]:
all_data_02 = all_data01[(all_data01['lat'] > 34.16985) & (all_data01['lat'] < 34.395586)\
           & (all_data01['lng'] > 108.756009) & (all_data01['lng'] < 109.146952)]

In [71]:
len(all_data_02)

163257

In [72]:
all_data_02.to_csv(r'../../file/subject/chapter5/all_data_2.csv')

### 3.坐标转换，BD09转WGS84

长文件，在IDE里写

In [3]:
data = pd.read_csv(r'../../file/subject/chapter5/检查.csv')

In [4]:
data[:5]

,index,name,lat,lng,sort1,sort2
0,0,爱骑行爱生活单车运动协作会,34.170078,108.831835,政府机构,党派团体
1,0,西安市教育考试中心,34.317549,108.944790,政府机构,政治教育机构
2,1,西安市教育考试中心,34.243562,108.958364,政府机构,政治教育机构
3,2,西安教育,34.312222,109.124069,政府机构,政治教育机构
4,3,教育培训基地,34.306449,109.066230,政府机构,政治教育机构


In [73]:
data['lat_wgs84'] = 0.00
data['lng_wgs84'] = 0.00
data.head()

,index,name,lat,lng,sort1,sort2,lat_wgs84,lng_wgs84
0,0,爱骑行爱生活单车运动协作会,34.170078,108.831835,政府机构,党派团体,0.0,0.0
1,0,西安市教育考试中心,34.317549,108.944790,政府机构,政治教育机构,0.0,0.0
2,1,西安市教育考试中心,34.243562,108.958364,政府机构,政治教育机构,0.0,0.0
3,2,西安教育,34.312222,109.124069,政府机构,政治教育机构,0.0,0.0
4,3,教育培训基地,34.306449,109.066230,政府机构,政治教育机构,0.0,0.0


In [7]:
data02 = data.copy()

In [74]:
#定义一个函数，能够修改pandas对象的值

In [20]:
len(data02)%100

45

In [58]:
for i in range(len(data02)-(len(data02)%100),len(data02)):
    data02.loc[i:i,('lat_wgs84','lng_wgs84')] = [i-1,i+1]

In [71]:
data02[3000:3003]

,index,name,lat,lng,sort1,sort2,lat_wgs84,lng_wgs84
3000,2,谭家街道文学艺术界联合会,34.340529,108.984108,政府机构,社会团体,2999,3001.0
3001,3,陕西省国际标准舞学会,34.353181,108.974071,政府机构,社会团体,3000,3002.0
3002,4,西安市未央区张家堡街道办事处五龙汤社区老年协会,34.355846,108.978418,政府机构,社会团体,3001,3003.0


In [11]:
import time
#前100整数个对象
start_time = time.time()
for i in range(29,31):
    def coor_trans():
        for j in  range(i*100):
            data02.loc[j:j,('lat_wgs84','lng_wgs84')] = [j-2,j+2]
    coor_trans()
#最后一个非100整数对象
for i in range(len(data02)-(len(data02)%100),len(data02)):
    data02.loc[i:i,('lat_wgs84','lng_wgs84')] = [i-2,i+2]
end_time = time.time()
print('time use:{}'.format(end_time-start_time))

time use:7.165005207061768


In [12]:
data02.loc[2995:3003]

,index,name,lat,lng,sort1,sort2,lat_wgs84,lng_wgs84
2995,5,谭家街道文学艺术界联合会,34.340529,108.984108,政府机构,党派团体,2993.0,2997.0
2996,6,中共五龙汤社区支部委员会,34.356650,108.975046,政府机构,党派团体,2994.0,2998.0
2997,7,西安市未央区张家堡街道办事处五龙汤社区老年协会,34.355846,108.978418,政府机构,党派团体,2995.0,2999.0
2998,8,中国共产党西安未央康乐医院支部委员会,34.352732,108.971605,政府机构,党派团体,2996.0,3000.0
2999,1,西安市示范基层老年协会,34.356742,108.975105,政府机构,社会团体,2997.0,3001.0
3000,2,谭家街道文学艺术界联合会,34.340529,108.984108,政府机构,社会团体,2998.0,3002.0
3001,3,陕西省国际标准舞学会,34.353181,108.974071,政府机构,社会团体,2999.0,3003.0
3002,4,西安市未央区张家堡街道办事处五龙汤社区老年协会,34.355846,108.978418,政府机构,社会团体,3000.0,3004.0
3003,5,航空行业第十三职业技能鉴定站操作技能培训考试基地,34.357061,108.972291,政府机构,社会团体,3001.0,3005.0


In [21]:
data02['lat'][3000]

34.340529

#### 将两次转换结果进行合并

In [85]:
df = pd.read_csv(r'../../file/subject/chapter5/all_wgs84_2.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,index,name,lat,lng,sort1,sort2,lat_wgs84,lng_wgs84
0,0,0,0,丰泽科技园-2号楼,34.176428,108.816795,房地产,写字楼,34.169933,108.810814
1,1,1,1,高新新区,34.173326,108.827928,房地产,写字楼,34.166777,108.821960
2,2,2,0,丰泽科技园-5号楼,34.175525,108.817244,房地产,住宅区,34.169014,108.811268
3,3,3,1,丰泽科技路3号楼,34.175989,108.817210,房地产,住宅区,34.169486,108.811232
4,4,4,2,鑫垚新区,34.174688,108.815573,房地产,住宅区,34.168163,108.809602


In [195]:
len(df)

163257

In [89]:
df2 = pd.read_csv(r'../../file/subject/chapter5/all_wgs84.csv')
df2.head()

,Unnamed: 0,Unnamed: 0.1,index,name,lat,lng,sort1,sort2,lat_wgs84,lng_wgs84
0,0,0,0,丰泽科技园-2号楼,34.176428,108.816795,房地产,写字楼,34.169933,108.810814
1,1,1,1,高新新区,34.173326,108.827928,房地产,写字楼,34.166777,108.821960
2,2,2,0,丰泽科技园-5号楼,34.175525,108.817244,房地产,住宅区,34.169014,108.811268
3,3,3,1,丰泽科技路3号楼,34.175989,108.817210,房地产,住宅区,34.169486,108.811232
4,4,4,2,鑫垚新区,34.174688,108.815573,房地产,住宅区,34.168163,108.809602


In [196]:
len(df2)

140146

In [93]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'index', 'name', 'lat', 'lng', 'sort1',
       'sort2', 'lat_wgs84', 'lng_wgs84'],
      dtype='object')

In [207]:
df[df.lat_wgs84 == 0]

,Unnamed: 0,Unnamed: 0.1,index,name,lat,lng,sort1,sort2,lat_wgs84,lng_wgs84
6,6,6,0,3号院住宅楼-4号楼,34.278397,109.006899,房地产,内部楼栋,0.0,0.0
7,7,7,1,北院门-158号楼,34.268386,108.949779,房地产,内部楼栋,0.0,0.0
8,8,8,2,北院门-55号楼,34.267452,108.949956,房地产,内部楼栋,0.0,0.0
9,9,9,3,北院门-202号楼,34.269206,108.949990,房地产,内部楼栋,0.0,0.0
10,10,10,4,北院门-108号楼,34.267762,108.949932,房地产,内部楼栋,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
162244,162244,7907,1,西安国际港务区行政审批服务局,34.384616,109.076859,政府机构,行政单位,0.0,0.0
162245,162245,7908,2,西安市城市管理综合行政执法局国际港务区分局,34.384409,109.080262,政府机构,行政单位,0.0,0.0
162246,162246,7909,3,西安国际港务区管委会征迁安置协调服务办公室,34.384238,109.079422,政府机构,行政单位,0.0,0.0
162247,162247,7916,0,西安市灞桥区新筑街道办事处新寇村民委员会,34.392327,109.092758,政府机构,各级政府,0.0,0.0


In [219]:
df_all = df.merge(df2,on=[ 'name', 'lat', 'lng', 'sort1','sort2'],how='outer')
len(df_all)

169876

In [225]:
df_all.drop_duplicates(subset=['name', 'lat', 'lng'],inplace=True)

In [226]:
df_all.columns

Index(['Unnamed: 0_x', 'Unnamed: 0.1_x', 'index_x', 'name', 'lat', 'lng',
       'sort1', 'sort2', 'lat_wgs84_x', 'lng_wgs84_x', 'Unnamed: 0_y',
       'Unnamed: 0.1_y', 'index_y', 'lat_wgs84_y', 'lng_wgs84_y'],
      dtype='object')

In [233]:
df_all01 = df_all[[ 'name', 'lat', 'lng',
       'sort1', 'sort2', 'lat_wgs84_x', 'lng_wgs84_x', 'lat_wgs84_y', 'lng_wgs84_y']]

In [235]:
df_all02 = df_all01.copy()

In [236]:
df_all02.drop_duplicates(subset=['name', 'lat', 'lng'],inplace=True)

In [237]:
len(df_all02)

169775

#### 填充空值

In [241]:
df_all02.fillna(0,inplace=True)

In [242]:
df_all02

,name,lat,lng,sort1,sort2,lat_wgs84_x,lng_wgs84_x,lat_wgs84_y,lng_wgs84_y
0,丰泽科技园-2号楼,34.176428,108.816795,房地产,写字楼,34.169933,108.810814,34.169933,108.810814
1,高新新区,34.173326,108.827928,房地产,写字楼,34.166777,108.821960,34.166777,108.821960
2,丰泽科技园-5号楼,34.175525,108.817244,房地产,住宅区,34.169014,108.811268,34.169014,108.811268
3,丰泽科技路3号楼,34.175989,108.817210,房地产,住宅区,34.169486,108.811232,34.169486,108.811232
4,鑫垚新区,34.174688,108.815573,房地产,住宅区,34.168163,108.809602,34.168163,108.809602
...,...,...,...,...,...,...,...,...,...
169871,渭阳街道办事处滨河社区居民委员会,34.354146,108.754023,政府机构,居民委员会,0.000000,0.000000,34.347680,108.747991
169872,中共金家庄村支部委员会(泰禾馨苑西南),34.352485,108.753392,政府机构,居民委员会,0.000000,0.000000,34.346047,108.747352
169873,李家堡村委会,34.361152,108.754856,政府机构,各级政府,0.000000,0.000000,34.354567,108.748862
169874,咸阳市渭城区渭阳街道李家堡社区居民监督委员会,34.363030,108.753303,政府机构,行政单位,0.000000,0.000000,34.356415,108.747319


In [249]:
df_all02.lat_wgs84_x = df_all02.apply(lambda x:'%.6f'%x['lat_wgs84_y'] if x['lat_wgs84_x'] == 0 else x.lat_wgs84_x, axis=1)
df_all02.lng_wgs84_x = df_all02.apply(lambda x:'%.6f'%x['lng_wgs84_y'] if x['lng_wgs84_x'] == 0 else x.lng_wgs84_x, axis=1)

In [ ]:
df_all02.drop(['lat_wgs84_y','lng_wgs84_y'],inplace=True,axis=1)

In [261]:
df_all02.rename(columns={'lat_wgs84_x':'lat_wgs84','lng_wgs84_x':'lng_wgs84'},inplace=True)

In [267]:
df_all02.to_csv(r'../../file/subject/chapter5/all_wgs84_3.csv')